<a href="https://colab.research.google.com/github/ehsan-lari/pyro101/blob/main/pyro_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Setup

In [ ]:
!pip install -q pyro-ppl

In [ ]:
import pyro
import pyro.distributions as dist

import torch

In [ ]:
normal = dist.Normal(
    torch.tensor([0.0, 1.0, 2.0]),
    1.0)

In [ ]:
# torch.manual_seed(42)

normal.sample()

In [ ]:
normal2 = dist.Normal(0, 1)

In [ ]:
normal2.sample((1, 3, 2))

### 1. Introducing Models 1.0

In [ ]:
def model(mu = 15.0, sigma = 2.0):
  temp = pyro.sample('temp', dist.Normal(mu, sigma))
  return temp

print(model())

In [ ]:
pyro.render_model(model, render_distributions=True)

### 2. Introducing Models 2.0

In [ ]:
def model2(mu = 15.0, sigma = 2.0):
  temp = pyro.sample('temp', dist.Normal(mu, sigma))
  sensor = pyro.sample('sensor', dist.Normal(temp, 1.0))
  return (temp, sensor)

print(model2())

In [ ]:
pyro.render_model(model2, render_distributions=True)

### 3. Doing Inference

#### 3.1 Model 1

In [ ]:
obs1 = {'sensor': torch.tensor([18.0])}

def model3(obs, mu = 15.0, sigma = 2.0):
  temp = pyro.sample('temp', dist.Normal(15.0, 2.0))
  sensor = pyro.sample('sensor', dist.Normal(temp, 0.1), obs=obs1['sensor'])

In [ ]:
pyro.render_model(model3, model_args=(obs1, ), render_distributions=True)

In [57]:
# import pandas as pd

# pd.DataFrame(obs)

#### 3.2 Model 2

In [58]:
def model4(obs):
  mean_temp = pyro.param('mean_temp', torch.tensor(15.0))
  # sigma_temp = pyro.param('sigma_temp', torch.tensor(2.0), constraint=dist.constraints.positive)
  for i in range(obs['sensor'].shape[0]):
    temp = pyro.sample(f'temp_{i}', dist.Normal(mean_temp, 2.0))
    sensor = pyro.sample(f'sensor_{i}', dist.Normal(temp, 1.0), obs=obs['sensor'][i])

In [ ]:
obs2 = {'sensor': 5*torch.rand(10) + 15.0}

pyro.render_model(model4, model_args=(obs2, ), render_params=True)

#### 3.3 Model 3

In [61]:
def model5(obs):
  mean_temp = pyro.sample('mean_temp', dist.Normal(15.0, 2.0))
  # sigma_temp = pyro.param('sigma_temp', torch.tensor(2.0), constraint=dist.constraints.positive)
  for i in range(obs['sensor'].shape[0]):
    temp = pyro.sample(f'temp_{i}', dist.Normal(mean_temp, 2.0))
    sensor = pyro.sample(f'sensor_{i}', dist.Normal(temp, 1.0), obs=obs['sensor'][i])

In [ ]:
pyro.render_model(model5, model_args=(obs2, ), render_params=True)

#### 3.4 Model 4

In [65]:
def model6(obs):
  mean_temp = pyro.sample('mean_temp', dist.Normal(15.0, 2.0))
  # sigma_temp = pyro.param('sigma_temp', torch.tensor(2.0), constraint=dist.constraints.positive)
  with pyro.plate(f"N={obs['sensor'].shape[0]}", obs['sensor'].shape[0]):
    temp = pyro.sample(f'temp', dist.Normal(mean_temp, 2.0))
    sensor = pyro.sample(f'sensor', dist.Normal(temp, 1.0), obs=obs['sensor'])

In [ ]:
pyro.render_model(model6, model_args=(obs2, ), render_distributions=True)

### 4. Icecream Shop

In [69]:
obs3 = {'sensor': 10*torch.rand(10) + 15.0,
        'sales': 20*torch.rand(10) + 50.0}

# import pandas as pd

# pd.DataFrame(obs3)

In [71]:
def model7(obs):
  mean_temp = pyro.sample('mean_temp', dist.Normal(15.0, 2.0))
  alpha = pyro.sample('alpha', dist.Normal(0, 100.0))
  beta = pyro.sample('beta', dist.Normal(0, 100.0))

  with pyro.plate(f"N={obs['sensor'].shape[0]}", obs['sensor'].shape[0]):
    temp = pyro.sample(f'temp', dist.Normal(mean_temp, 2.0))
    sensor = pyro.sample(f'sensor', dist.Normal(temp, 1.0), obs=obs['sensor'])
    rate = torch.maximum(torch.tensor(1e-3), alpha + beta * temp)
    sales = pyro.sample(f'sales', dist.Poisson(rate), obs=obs['sales'])

In [ ]:
pyro.render_model(model7, model_args=(obs3, ), render_distributions=True)

In [81]:
obs4 = {'sensor': 10*torch.rand(100) + 15.0,
        'sales': 20*torch.rand(100) + 50.0,
        'sensor_humidity': 40*torch.rand(100) + 60.0}

In [82]:
def model8(obs):
  mean_temp = pyro.sample('mean_temp', dist.Normal(15.0, 2.0))
  mean_humidity = pyro.sample('mean_humidity', dist.Normal(80.0, 10.0))
  alpha = pyro.sample('alpha', dist.Normal(0, 100.0))
  beta = pyro.sample('beta', dist.Normal(0, 100.0))
  gamma = pyro.sample('gamma', dist.Normal(0, 100.0))

  with pyro.plate(f"N={obs['sensor'].shape[0]}", obs['sensor'].shape[0]):
    temp = pyro.sample(f'temp', dist.Normal(mean_temp, 2.0))
    sensor = pyro.sample(f'sensor', dist.Normal(temp, 1.0), obs=obs['sensor'])
    humidity = pyro.sample(f'humidity', dist.Normal(mean_humidity, 10.0))
    sensor_humidity = pyro.sample(f'sensor_humidity', dist.Normal(humidity, 5.0), obs=obs['sensor_humidity'])
    rate = torch.maximum(torch.tensor(1e-3), alpha + beta * temp + gamma * humidity)
    sales = pyro.sample(f'sales', dist.Poisson(rate), obs=obs['sales'])

In [ ]:
pyro.render_model(model8, model_args=(obs4, ), render_distributions=True)